Detectron2 Installations

In [1]:
!pip install pyyaml==5.1
# This is the current pytorch version on Colab. Uncomment this if Colab changes its pytorch version
!pip install torch==1.9.0+cu111 torchvision==0.10.0+cu111 -f https://download.pytorch.org/whl/torch_stable.html

# Install detectron2 that matches the above pytorch version
# See https://detectron2.readthedocs.io/tutorials/install.html for instructions
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu111/torch1.9/index.html
# exit(0)  # After installation, you need to "restart runtime" in Colab. This line can also restart runtime

Looking in links: https://download.pytorch.org/whl/torch_stable.html
Looking in links: https://dl.fbaipublicfiles.com/detectron2/wheels/cu111/torch1.9/index.html


In [2]:
# check pytorch installation: 
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
assert torch.__version__.startswith("1.9")   # please manually install torch 1.9 if Colab changes its default version

1.9.0+cu111 True


In [3]:
# You may need to restart your runtime prior to this, to let your installation take effect
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import cv2
import random


# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

Connecting Drive with colab

In [4]:
# ! conda install -y gdown


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Dataloader part of detectron2 for better understanding read example doc file - https://colab.research.google.com/drive/16jcaJoc6bCFAQ96jDe2HwtXj7BMD_-m5
and  this for object detection - https://gilberttanner.com/blog/train-a-microcontroller-detector-using-detectron2

In [6]:
!cp -r "/content/drive/MyDrive/Colab Notebooks/Barcode Reading and Decoding/Barcode and Partial/Dataset" "/content"
# # processed.zip
# !gdown --id 11kKI0U9cmcMOsezPSWEg4eK_6p3XvKiI

# # validation full_data.json
# !gdown --id 1Roh3aeo5inByOquFvM4HEw1Zx3YtXusv

In [7]:
# !unzip -qq "./processed.zip"

# !unzip -qq "./val.zip"

# !rm "./val/full_data.json"

import os 

print(len(os.listdir("/content/Dataset/train")))
print(len(os.listdir("/content/Dataset/val")))

14
5


In [8]:
import os
import numpy as np
import json
from detectron2.structures import BoxMode

def get_table_dicts(img_dir,d):
    if d == 'val':
        json_file = "/content/Dataset/val/demo.json"
    else:
        json_file = "/content/Dataset/train/demo.json"
    
    f =  open(json_file,) 
    imgs_anns = json.load(f)

    dataset_dicts = []
    for idx, v in enumerate(imgs_anns.values()):
        #print(v)
        record = {}
        
        filename = os.path.join(img_dir, v["filename"])
        height, width = cv2.imread(filename).shape[:2]
        
        record["file_name"] = filename
        record["image_id"] = idx
        record["height"] = height
        record["width"] = width
      
        annos = v["regions"]
        objs = []
        for anno in annos:
            typeofbarcode = anno["region_attributes" ]["names"]
            anno = anno["shape_attributes"]
            px = anno["all_points_x"]
            py = anno["all_points_y"]
            poly = [(x + 0.5, y + 0.5) for x, y in zip(px, py)]
            poly = [p for x in poly for p in x]
            if typeofbarcode == "complete":
              class_id = 0
            else:
              class_id = 1
            obj = {
                "bbox": [np.min(px), np.min(py), np.max(px), np.max(py)],
                "bbox_mode": BoxMode.XYXY_ABS,
                "segmentation": [poly],
                "category_id": int(class_id),
                "iscrowd": 0
            }
            objs.append(obj)
        record["annotations"] = objs
        dataset_dicts.append(record)
    return dataset_dicts



In [9]:
# "15-09-52-253_radiometric.png" in os.listdir("./content/processed/val")

In [10]:
# import matplotlib.pyplot as plt
# img_dir = "./content/processed/val"

# json_file = "./full_data.json"
    
# f =  open(json_file,) 
# imgs_anns = json.load(f)

# dataset_dicts = []

# for idx, v in enumerate(imgs_anns.values()):
#     if not v["filename"] in os.listdir("./content/processed/val"):
#         print(v["filename"])
#         print(v["filename"] in os.listdir("./content/processed/val"))
#     filename = os.path.join(img_dir, v["filename"])
    
# #     img = cv2.imread(filename)
# #     print(type(img))
# #     plt.imshow(img)
    

One of the most important part while training models with detectron2 is reistering the Dataset and creating a metadata for the dataset it helps in train and also in inferencing the model for better understanding you can look at - https://colab.research.google.com/drive/16jcaJoc6bCFAQ96jDe2HwtXj7BMD_-m5 

In [11]:
from detectron2.data import DatasetCatalog, MetadataCatalog
for d in ["train", "val"]: #, 
    DatasetCatalog.register("table_" + d, lambda d=d: get_table_dicts("/content/Dataset/" + d,d))
    MetadataCatalog.get("table_" + d).set(thing_classes=["complete","partial"])
table_metadata = MetadataCatalog.get("table_train")

Save the stored metadata in a file so that we can use it for inferencing I have used pickle to store my metadata you can use whatever you want and make sure that you have stored the pickle folder in an appropriate location.

In [12]:
import pickle 
# dbfile = open("table_full_data.pkl", "wb")
with open('table_full_data.pkl', 'wb') as file:
      
    # A new file will be created
    pickle.dump(table_metadata, file)

Helps in computation validation loss you can copy past next two cells 

In [13]:
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
from detectron2.engine.hooks import HookBase
from detectron2.evaluation import inference_context
from detectron2.utils.logger import log_every_n_seconds
from detectron2.data import DatasetMapper, build_detection_test_loader
import detectron2.utils.comm as comm
import torch
import time
import datetime
import logging

class LossEvalHook(HookBase):
    def __init__(self, eval_period, model, data_loader):
        self._model = model
        self._period = eval_period
        self._data_loader = data_loader
    
    def _do_loss_eval(self):
        # Copying inference_on_dataset from evaluator.py
        total = len(self._data_loader)
        num_warmup = min(5, total - 1)
            
        start_time = time.perf_counter()
        total_compute_time = 0
        losses = []
        for idx, inputs in enumerate(self._data_loader):            
            if idx == num_warmup:
                start_time = time.perf_counter()
                total_compute_time = 0
            start_compute_time = time.perf_counter()
            if torch.cuda.is_available():
                torch.cuda.synchronize()
            total_compute_time += time.perf_counter() - start_compute_time
            iters_after_start = idx + 1 - num_warmup * int(idx >= num_warmup)
            seconds_per_img = total_compute_time / iters_after_start
            if idx >= num_warmup * 2 or seconds_per_img > 5:
                total_seconds_per_img = (time.perf_counter() - start_time) / iters_after_start
                eta = datetime.timedelta(seconds=int(total_seconds_per_img * (total - idx - 1)))
                log_every_n_seconds(
                    logging.INFO,
                    "Loss on Validation  done {}/{}. {:.4f} s / img. ETA={}".format(
                        idx + 1, total, seconds_per_img, str(eta)
                    ),
                    n=5,
                )
            loss_batch = self._get_loss(inputs)
            losses.append(loss_batch)
        mean_loss = np.mean(losses)
        self.trainer.storage.put_scalar('validation_loss', mean_loss)
        comm.synchronize()

        return losses
            
    def _get_loss(self, data):
        # How loss is calculated on train_loop 
        metrics_dict = self._model(data)
        metrics_dict = {
            k: v.detach().cpu().item() if isinstance(v, torch.Tensor) else float(v)
            for k, v in metrics_dict.items()
        }
        total_losses_reduced = sum(loss for loss in metrics_dict.values())
        return total_losses_reduced
        
        
    def after_step(self):
        next_iter = self.trainer.iter + 1
        is_final = next_iter == self.trainer.max_iter
        if is_final or (self._period > 0 and next_iter % self._period == 0):
            self._do_loss_eval()
        self.trainer.storage.put_scalars(timetest=12)

In [14]:
from detectron2.engine import DefaultTrainer, default_argument_parser, default_setup, hooks, launch
class MyTrainer(DefaultTrainer):
    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
     #   if output_folder is None:
     #       output_folder = os.path.join(cfg.OUTPUT_DIR, "inference")
        return COCOEvaluator(dataset_name, cfg, False, output_dir="./output/")
                     
    def build_hooks(self):
        hooks = super().build_hooks()
        hooks.insert(-1,LossEvalHook(
            cfg.TEST.EVAL_PERIOD,
            self.model,
            build_detection_test_loader(
                self.cfg,
                self.cfg.DATASETS.TEST[0],
                DatasetMapper(self.cfg,True)
            )
        ))
        return hooks

In [15]:
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader

import os

Model training

In [ ]:
cfg = get_cfg()
cfg.DATALOADER.FILTER_EMPTY_ANNOTATIONS=False
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("table_train",)
cfg.DATASETS.TEST = ("table_val",)
cfg.TEST.EVAL_PERIOD = 999 #evaluates map value after every 1000 iterations

cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml")  # Let training initialize from model zoo
# cfg.MODEL.WEIGHTS =  './output/model_0014999.pth'  #if you want to resume training you have to give the last weight file 
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025  # pick a good LR
# cfg.SOLVER.STEPS = 210000
cfg.SOLVER.MAX_ITER = 5000  
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512   
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 2  
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True) #creates an output directory
# trainer = DefaultTrainer(cfg)      if you want to train your model without validation loss you can use these commands
# trainer.resume_or_load(resume=False)
# trainer.train()


trainer = MyTrainer(cfg)
trainer.resume_or_load(resume=True)    #if you want to resume training you model resume = True
if cfg.TEST.AUG.ENABLED:
  trainer.build_evaluator(cfg, "table_val")
  trainer.build_hooks()

trainer.train()

[04/30 10:38:16 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (3, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (3,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 1024) in the checkpoint but (8, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (8,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.mask_head.predictor.weight' to the model due to incompatible shapes: (80, 256, 1, 1) in the checkpoint but (2, 256, 1, 1) in

[04/30 10:38:26 d2.engine.train_loop]: Starting training from iteration 0


/usr/local/lib/python3.7/dist-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


[04/30 10:39:04 d2.utils.events]:  eta: 2:30:41  iter: 19  total_loss: 2.309  loss_cls: 1.208  loss_box_reg: 0.06609  loss_mask: 0.6905  loss_rpn_cls: 0.3387  loss_rpn_loc: 0.03883  time: 1.8310  data_time: 0.0905  lr: 4.9953e-06  max_mem: 3235M
[04/30 10:39:40 d2.utils.events]:  eta: 2:33:17  iter: 39  total_loss: 2.329  loss_cls: 1.049  loss_box_reg: 0.09185  loss_mask: 0.6858  loss_rpn_cls: 0.5094  loss_rpn_loc: 0.04566  time: 1.8277  data_time: 0.0150  lr: 9.9902e-06  max_mem: 3235M
[04/30 10:40:16 d2.utils.events]:  eta: 2:32:36  iter: 59  total_loss: 1.92  loss_cls: 0.7675  loss_box_reg: 0.07247  loss_mask: 0.6752  loss_rpn_cls: 0.29  loss_rpn_loc: 0.03558  time: 1.8174  data_time: 0.0128  lr: 1.4985e-05  max_mem: 3235M
[04/30 10:40:54 d2.utils.events]:  eta: 2:32:03  iter: 79  total_loss: 1.638  loss_cls: 0.5551  loss_box_reg: 0.1538  loss_mask: 0.6593  loss_rpn_cls: 0.165  loss_rpn_loc: 0.03833  time: 1.8330  data_time: 0.0129  lr: 1.998e-05  max_mem: 3235M
[04/30 10:41:31 d2.u

In [ ]:
# import tensorflow as tf
# import datetime, os
# # Look at training curves in tensorboard:
# %load_ext tensorboard
# %tensorboard --logdir output

In [ ]:
!ls "./output"

In [ ]:
# Pick the file name from above ls command whether model_final.pth adn model_0034999.pth

from IPython.display import FileLink
# FileLink(r'./output/model_final.pth')


In [ ]:
# FileLink(r'./output/model_0034999.pth')

In [ ]:
FileLink(r'table_full_data.pkl')

In [ ]:
f = open("/content/Dataset/train/demo.json",)

data = json.load(f)

In [ ]:
data